In [1]:
import pandas as pd
import tensorflow as tf
import keras as K
import numpy as np
import pylab as plt
%matplotlib inline

def precision(y_true, y_pred):

    # Count positive samples.
    c1 = K.backend.sum(K.backend.round(K.backend.clip(y_true * y_pred, 0, 1)))
    c2 = K.backend.sum(K.backend.round(K.backend.clip(y_pred, 0, 1)))

    # How many selected items are relevant?
    return c1 / c2

def recall(y_true, y_pred):

    # Count positive samples.
    c1 = K.backend.sum(K.backend.round(K.backend.clip(y_true * y_pred, 0, 1)))
    c3 = K.backend.sum(K.backend.round(K.backend.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0
    # How many relevant items are selected?
    return c1 / c3

def f1(y_true, y_pred):
    recall_score = recall(y_true, y_pred)
    precision_score = precision(y_true, y_pred)
    # Calculate f1_score
    return 2 * (precision_score * recall_score) / (precision_score + recall_score)

def plot_history(history, filename):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    f1_list = [s for s in history.history.keys() if 'f1' in s and 'val' not in s]
    val_f1_list = [s for s in history.history.keys() if 'f1' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    fig = plt.figure(figsize=(15,5))
    plt.subplot(131)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## F1
    plt.subplot(132)
    for l in f1_list:
        plt.plot(epochs, history.history[l], 'b', label='Training F1 (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_f1_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation F1 (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('F1')
    plt.xlabel('Epochs')
    plt.ylabel('F1')
    plt.legend()
    
    ## Accuracy
    plt.subplot(133)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training Acc (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation Acc (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Acc')
    plt.xlabel('Epochs')
    plt.ylabel('Acc')
    plt.legend()
    fig.savefig(filename)

Using TensorFlow backend.


In [2]:
model = K.applications.ResNet50(weights = "imagenet", include_top=False, input_shape = (img_width,img_height, 3))
#Display Model's architecture
model.summary()

NameError: name 'img_width' is not defined

In [2]:
# Load training and validation data
balanced_training_data = pd.read_csv('../../data/smote_training_data.csv', index_col=0)
validation_data = pd.read_csv('../../data/validation_data.csv', index_col=0)

nb_classes = len(balanced_training_data.Class.unique())
img_width, img_height = 128, 128
train_data_dir = "../../data/train/"
validation_data_dir = "../../data/validation/"

batch_size = 32

# ResNet50 pretrained model
model = K.applications.ResNet50(weights = "imagenet", include_top=False, input_shape = (img_width,img_height, 3))
#Display Model's architecture
# model.summary()

# Dropout ratio
dropout_ratio = 0.2

# Freeze the layers which you don't want to train. 
unfrozen_layers=0
for layer in model.layers[:len(model.layers)-unfrozen_layers]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = K.layers.Flatten()(x)
# x = K.layers.Dense(1024, activation="relu")(x)
# x = K.layers.Dropout(dropout_ratio)(x)
# x = K.layers.Dense(1024, activation="relu")(x)
predictions = K.layers.Dense(nb_classes, activation="softmax")(x)

# creating the final model 
model_final = K.models.Model(input = model.input, output = predictions)

# compile the model 
loss_rate = 0.001
model_final.compile(loss = "categorical_crossentropy", optimizer = K.optimizers.Adam(lr=loss_rate), metrics=['accuracy', f1])

InternalError: Failed to create session.

In [ ]:
# Use Data Augmentation
data_augmentation = False

print('Data augmentation in use? {}.'.format(data_augmentation))
if not data_augmentation:
    train_datagen = K.preprocessing.image.ImageDataGenerator(rescale = 1./255)
else:
    # Initiate the train and validation generators with data Augumentation 
    train_datagen = K.preprocessing.image.ImageDataGenerator(rescale = 1./255, 
                                                             horizontal_flip = False, 
                                                             vertical_flip = False,
                                                             fill_mode = 'nearest', 
                                                             zoom_range = 0,
                                                             width_shift_range = 0, 
                                                             height_shift_range = 0, 
                                                             rotation_range=360)

validation_datagen = K.preprocessing.image.ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(train_data_dir,target_size = (img_height, img_width),batch_size = batch_size, class_mode = "categorical")
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,target_size = (img_height, img_width),class_mode = "categorical")

nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)

In [ ]:
# Save the model according to the conditions  
# checkpoint = K.callbacks.ModelCheckpoint("{}.h5".format(model.name), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = K.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

epochs=5
filename = '{}_last_{}_unfrozen_lr{}_{}_data_augmentation'.format(model.name, unfrozen_layers, str(loss_rate).replace('.', ''), 'with' if 
                                     data_augmentation else 'without')

print('Tensorboard records available under: {}'.format(filename))
tensorboard = K.callbacks.TensorBoard(log_dir='./logs/{}'.format(filename),
                                     histogram_freq=0, write_graph=True, write_images=True)

history = model_final.fit_generator(train_generator,train_generator.n // batch_size, epochs=epochs, workers=4,
    validation_data=validation_generator, callbacks=[tensorboard],validation_steps=validation_generator.n // batch_size)

plot_history(history, '{}.png'.format(filename))